# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [3]:
# Write your code below.

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [125]:
import os
from glob import glob

# Write your code below.

price_data_path = os.getenv('PRICE_DATA')

parq_files_paths_list = glob(price_data_path+'/**/part.*.parquet', recursive=True)

#print(parq_files_paths_list)
#print(len(parq_files_paths_list))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [89]:
# Write your code below.

df = dd.read_parquet(parq_files_paths_list).set_index('ticker')
#print(df.head())

dd_feat = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag_1 = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

/var/folders/j2/psnwl2cs3k165th6cmnf1nr40000gn/T/ipykernel_97469/4086497202.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [113]:
# Write your code below.

pd_df = dd_feat.compute()

In [114]:
print(pd_df)

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
OXY    2023-01-03  62.299999  62.950001  60.110001  61.049999  59.891590   
OXY    2023-01-04  59.939999  61.840000  59.700001  61.110001  59.950447   
OXY    2023-01-05  61.119999  62.730000  60.750000  62.220001  61.039387   
OXY    2023-01-06  63.410000  64.629997  62.939999  63.750000  62.540356   
OXY    2023-01-09  64.870003  66.580002  63.980000  64.080002  62.864094   
...           ...        ...        ...        ...        ...        ...   
INTU   2014-12-24  95.300003  95.430000  94.580002  94.589996  87.320663   
INTU   2014-12-26  94.519997  94.949997  94.110001  94.190002  86.951416   
INTU   2014-12-29  94.080002  94.459999  93.730003  93.760002  86.554459   
INTU   2014-12-30  93.169998  94.040001  92.639999  93.110001  85.954384   
INTU   2014-12-31  93.180000  93.889999  92.160004  92.190002  85.105103   

          V

In [121]:
temp_pd_df = pd_df

temp_pd_df['returns mean'] = temp_pd_df['returns'].rolling(10).mean()

temp_pd_df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns mean
ticker,,,,,,,,,,,,,,,
OXY,2023-01-03,62.299999,62.950001,60.110001,61.049999,59.891590,13104000,Energy,Oil & Gas Exploration & Production,2023,NaN,NaN,NaN,2.840000,NaN
OXY,2023-01-04,59.939999,61.840000,59.700001,61.110001,59.950447,12238000,Energy,Oil & Gas Exploration & Production,2023,61.049999,59.891590,0.000983,2.139999,NaN
OXY,2023-01-05,61.119999,62.730000,60.750000,62.220001,61.039387,10692800,Energy,Oil & Gas Exploration & Production,2023,61.110001,59.950447,0.018164,1.980000,NaN
OXY,2023-01-06,63.410000,64.629997,62.939999,63.750000,62.540356,9832700,Energy,Oil & Gas Exploration & Production,2023,62.220001,61.039387,0.024590,1.689999,NaN
OXY,2023-01-09,64.870003,66.580002,63.980000,64.080002,62.864094,12519800,Energy,Oil & Gas Exploration & Production,2023,63.750000,62.540356,0.005176,2.600002,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2014-12-24,95.300003,95.430000,94.580002,94.589996,87.320663,352500,Information Technology,Application Software,2014,95.150002,87.837646,-0.005886,0.849998,0.002211
INTU,2014-12-26,94.519997,94.949997,94.110001,94.190002,86.951416,638600,Information Technology,Application Software,2014,94.589996,87.320663,-0.004229,0.839996,0.000913
INTU,2014-12-29,94.080002,94.459999,93.730003,93.760002,86.554459,678400,Information Technology,Application Software,2014,94.190002,86.951416,-0.004565,0.729996,0.001742


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?


**It wasn't necessary to convert to pandas to calculate the moving average return, and it would have been better to do it in Dask because Dask is designed for handling large datasets and can efficiently parallelize operations across multiple cores or machines. Dask's scalability, memory efficiency, and lazy evaluation make it ideal for big data applications. Additionally, its seamless integration with existing Python tools allows for efficient handling of large datasets while maintaining the ease of use familiar to pandas. Therefore, using Dask can provide significant performance and efficiency benefits for large-scale data analysis tasks.**

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.